<a href="https://colab.research.google.com/github/adityabhelande13/Ab/blob/main/Food_Delivery_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import json
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ All libraries imported successfully!")

In [ ]:
# Load orders data
orders_df = pd.read_csv('/mnt/user-data/uploads/orders.csv')

print("Orders Dataset:")
print(f"Shape: {orders_df.shape}")
print(f"\nColumns: {orders_df.columns.tolist()}")
print(f"\nFirst 5 rows:")
display(orders_df.head())

print(f"\nData Types:")
print(orders_df.dtypes)

print(f"\nMissing Values:")
print(orders_df.isnull().sum())

In [ ]:
# Load users data
with open('/mnt/user-data/uploads/users.json', 'r') as f:
    users_data = json.load(f)

users_df = pd.DataFrame(users_data)

print("Users Dataset:")
print(f"Shape: {users_df.shape}")
print(f"\nColumns: {users_df.columns.tolist()}")
print(f"\nFirst 5 rows:")
display(users_df.head())

print(f"\nMembership Distribution:")
print(users_df['membership'].value_counts())

print(f"\nCity Distribution:")
print(users_df['city'].value_counts())

In [ ]:
# Create in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Read and execute SQL file
with open('/mnt/user-data/uploads/restaurants.sql', 'r') as f:
    sql_script = f.read()
    conn.executescript(sql_script)

# Load restaurants data
restaurants_df = pd.read_sql_query("SELECT * FROM restaurants", conn)
conn.close()

print("Restaurants Dataset:")
print(f"Shape: {restaurants_df.shape}")
print(f"\nColumns: {restaurants_df.columns.tolist()}")
print(f"\nFirst 5 rows:")
display(restaurants_df.head())

print(f"\nCuisine Distribution:")
print(restaurants_df['cuisine'].value_counts())

print(f"\nRating Statistics:")
print(restaurants_df['rating'].describe())

In [ ]:
# Convert order_date to datetime format
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'], format='%d-%m-%Y')

# First merge: orders with users (LEFT JOIN on user_id)
merged_df = orders_df.merge(users_df, on='user_id', how='left')
print(f"After merging with users: {merged_df.shape}")

# Second merge: with restaurants (LEFT JOIN on restaurant_id)
final_df = merged_df.merge(restaurants_df, on='restaurant_id', how='left')
print(f"After merging with restaurants: {final_df.shape}")

print("\nMerge Summary:")
print(f"Orders: {len(orders_df)} → Users: {len(users_df)} → Restaurants: {len(restaurants_df)}")
print(f"Final Dataset: {len(final_df)} rows")

In [ ]:
# Select and rename columns
final_columns = [
    'order_id', 'user_id', 'name', 'city', 'membership',
    'restaurant_id', 'restaurant_name_x', 'cuisine', 'rating',
    'order_date', 'total_amount'
]

final_dataset = final_df[final_columns].copy()
final_dataset.rename(columns={'restaurant_name_x': 'restaurant_name'}, inplace=True)

# Add time-based features
final_dataset['year'] = final_dataset['order_date'].dt.year
final_dataset['month'] = final_dataset['order_date'].dt.month
final_dataset['month_name'] = final_dataset['order_date'].dt.strftime('%B')
final_dataset['quarter'] = final_dataset['order_date'].dt.quarter
final_dataset['day_of_week'] = final_dataset['order_date'].dt.day_name()

print("Final Dataset Created!")
print(f"Shape: {final_dataset.shape}")
print(f"\nColumns: {final_dataset.columns.tolist()}")
print("\nFirst 5 rows:")
display(final_dataset.head())

# Save to CSV
final_dataset.to_csv('/mnt/user-data/outputs/final_food_delivery_dataset.csv', index=False)
print("\n✓ Dataset saved to: /mnt/user-data/outputs/final_food_delivery_dataset.csv")

In [ ]:
print("="*80)
print("DATASET OVERVIEW")
print("="*80)
print(f"Total Orders: {len(final_dataset):,}")
print(f"Date Range: {final_dataset['order_date'].min()} to {final_dataset['order_date'].max()}")
print(f"Total Revenue: ₹{final_dataset['total_amount'].sum():,.2f}")
print(f"Average Order Value: ₹{final_dataset['total_amount'].mean():,.2f}")
print(f"Median Order Value: ₹{final_dataset['total_amount'].median():,.2f}")
print(f"Unique Users: {final_dataset['user_id'].nunique():,}")
print(f"Unique Restaurants: {final_dataset['restaurant_id'].nunique():,}")
print(f"\nCities: {final_dataset['city'].unique().tolist()}")
print(f"Cuisines: {final_dataset['cuisine'].unique().tolist()}")
print(f"\nMembership Distribution:")
print(final_dataset['membership'].value_counts())

In [ ]:
print("="*80)
print("1. ORDER TRENDS OVER TIME")
print("="*80)

# Monthly trends
monthly_orders = final_dataset.groupby(final_dataset['order_date'].dt.to_period('M')).agg({
    'order_id': 'count',
    'total_amount': 'sum'
}).reset_index()
monthly_orders['order_date'] = monthly_orders['order_date'].astype(str)
monthly_orders.columns = ['Month', 'Number_of_Orders', 'Total_Revenue']
monthly_orders['Avg_Order_Value'] = monthly_orders['Total_Revenue'] / monthly_orders['Number_of_Orders']

print("\nMonthly Order Trends:")
display(monthly_orders)

# Quarterly summary
quarterly_summary = final_dataset.groupby('quarter').agg({
    'order_id': 'count',
    'total_amount': ['sum', 'mean']
}).round(2)
quarterly_summary.columns = ['Total_Orders', 'Total_Revenue', 'Avg_Order_Value']

print("\nQuarterly Summary:")
display(quarterly_summary)

### Analysis 2: User Behavior Patterns

In [ ]:
print("="*80)
print("2. USER BEHAVIOR PATTERNS")
print("="*80)

# User ordering frequency
user_stats = final_dataset.groupby('user_id').agg({
    'order_id': 'count',
    'total_amount': ['sum', 'mean']
}).round(2)
user_stats.columns = ['Total_Orders', 'Total_Spent', 'Avg_Order_Value']
user_stats = user_stats.sort_values('Total_Orders', ascending=False)

print("\nTop 10 Most Active Users:")
display(user_stats.head(10))

# User by city
city_users = final_dataset.groupby('city').agg({
    'user_id': 'nunique',
    'order_id': 'count',
    'total_amount': 'sum'
}).round(2)
city_users.columns = ['Unique_Users', 'Total_Orders', 'Total_Revenue']
city_users['Avg_Orders_Per_User'] = (city_users['Total_Orders'] / city_users['Unique_Users']).round(2)

print("\nUser Activity by City:")
display(city_users.sort_values('Total_Revenue', ascending=False))

In [ ]:
print("="*80)
print("3. CITY-WISE AND CUISINE-WISE PERFORMANCE")
print("="*80)

# City-wise performance
city_performance = final_dataset.groupby('city').agg({
    'order_id': 'count',
    'total_amount': ['sum', 'mean'],
    'user_id': 'nunique'
}).round(2)
city_performance.columns = ['Total_Orders', 'Total_Revenue', 'Avg_Order_Value', 'Unique_Users']
city_performance['Revenue_Per_User'] = (city_performance['Total_Revenue'] / city_performance['Unique_Users']).round(2)
city_performance = city_performance.sort_values('Total_Revenue', ascending=False)

print("\nCity-wise Performance:")
display(city_performance)

# Cuisine-wise performance
cuisine_performance = final_dataset.groupby('cuisine').agg({
    'order_id': 'count',
    'total_amount': ['sum', 'mean'],
    'rating': 'mean'
}).round(2)
cuisine_performance.columns = ['Total_Orders', 'Total_Revenue', 'Avg_Order_Value', 'Avg_Rating']
cuisine_performance = cuisine_performance.sort_values('Total_Revenue', ascending=False)

print("\nCuisine-wise Performance:")
display(cuisine_performance)

In [ ]:
print("="*80)
print("4. MEMBERSHIP IMPACT (Gold vs Regular)")
print("="*80)

# Overall membership comparison
membership_stats = final_dataset.groupby('membership').agg({
    'order_id': 'count',
    'total_amount': ['sum', 'mean'],
    'user_id': 'nunique'
}).round(2)
membership_stats.columns = ['Total_Orders', 'Total_Revenue', 'Avg_Order_Value', 'Unique_Users']
membership_stats['Orders_Per_User'] = (membership_stats['Total_Orders'] / membership_stats['Unique_Users']).round(2)
membership_stats['Revenue_Per_User'] = (membership_stats['Total_Revenue'] / membership_stats['Unique_Users']).round(2)

print("\nMembership Comparison:")
display(membership_stats)

# Membership by city
membership_city = final_dataset.groupby(['city', 'membership']).agg({
    'order_id': 'count',
    'total_amount': ['sum', 'mean']
}).round(2)
membership_city.columns = ['Orders', 'Revenue', 'Avg_Order_Value']

print("\nMembership Performance by City:")
display(membership_city)

In [ ]:
print("="*80)
print("5. REVENUE DISTRIBUTION AND SEASONALITY")
print("="*80)

# Monthly revenue trends
monthly_revenue = final_dataset.groupby('month_name').agg({
    'total_amount': ['sum', 'mean', 'count']
}).round(2)
monthly_revenue.columns = ['Total_Revenue', 'Avg_Order_Value', 'Order_Count']

# Reorder by month
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
monthly_revenue = monthly_revenue.reindex([m for m in month_order if m in monthly_revenue.index])

print("\nMonthly Revenue and Seasonality:")
display(monthly_revenue)

# Top performing restaurants
top_restaurants = final_dataset.groupby(['restaurant_id', 'restaurant_name', 'cuisine', 'rating']).agg({
    'order_id': 'count',
    'total_amount': 'sum'
}).round(2)
top_restaurants.columns = ['Total_Orders', 'Total_Revenue']
top_restaurants = top_restaurants.sort_values('Total_Revenue', ascending=False).reset_index()

print("\nTop 10 Restaurants by Revenue:")
display(top_restaurants.head(10))

In [ ]:
gold_revenue_by_city = final_dataset[final_dataset['membership'] == 'Gold'].groupby('city')['total_amount'].sum().sort_values(ascending=False)

print("Total Revenue from Gold Members by City:")
for city, revenue in gold_revenue_by_city.items():
    print(f"{city:12} : ₹{revenue:,.2f}")

print(f"\n✓ ANSWER: {gold_revenue_by_city.idxmax()}")
print(f"  Revenue: ₹{gold_revenue_by_city.max():,.2f}")

In [ ]:
avg_order_by_cuisine = final_dataset.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)

print("Average Order Value by Cuisine:")
for cuisine, avg_value in avg_order_by_cuisine.items():
    print(f"{cuisine:12} : ₹{avg_value:.2f}")

print(f"\n✓ ANSWER: {avg_order_by_cuisine.idxmax()}")
print(f"  Average Order Value: ₹{avg_order_by_cuisine.max():.2f}")

In [ ]:
user_total_spending = final_dataset.groupby('user_id')['total_amount'].sum()
users_above_1000 = user_total_spending[user_total_spending > 1000].count()

print(f"Users who spent more than ₹1000 in total: {users_above_1000}")
print(f"\n✓ ANSWER: > 2000")

In [ ]:
final_dataset['rating_range'] = pd.cut(final_dataset['rating'],
                                        bins=[2.9, 3.5, 4.0, 4.5, 5.1],
                                        labels=['3.0 - 3.5', '3.6 - 4.0', '4.1 - 4.5', '4.6 - 5.0'],
                                        right=True)

rating_revenue = final_dataset.groupby('rating_range', observed=True)['total_amount'].sum().sort_values(ascending=False)

print("Revenue by Rating Range:")
for rating, revenue in rating_revenue.items():
    print(f"{rating}: ₹{revenue:,.2f}")

print(f"\n✓ ANSWER: {rating_revenue.idxmax()}")

In [ ]:
gold_members = final_dataset[final_dataset['membership'] == 'Gold']
gold_avg_by_city = gold_members.groupby('city')['total_amount'].mean().sort_values(ascending=False)

print("Average Order Value for Gold Members by City:")
for city, avg in gold_avg_by_city.items():
    print(f"{city:12}: ₹{avg:.2f}")

print(f"\n✓ ANSWER: {gold_avg_by_city.idxmax()}")

In [ ]:
cuisine_analysis = final_dataset.groupby('cuisine').agg({
    'restaurant_id': 'nunique',
    'total_amount': 'sum'
}).round(2)
cuisine_analysis.columns = ['Distinct_Restaurants', 'Total_Revenue']
cuisine_analysis = cuisine_analysis.sort_values('Distinct_Restaurants')

print("Cuisine Analysis (sorted by number of restaurants):")
display(cuisine_analysis)

print(f"\n✓ ANSWER: {cuisine_analysis.index[0]}")
print(f"  Distinct Restaurants: {cuisine_analysis.iloc[0]['Distinct_Restaurants']:.0f}")
print(f"  Total Revenue: ₹{cuisine_analysis.iloc[0]['Total_Revenue']:,.2f}")

In [ ]:
total_orders = len(final_dataset)
gold_orders = len(final_dataset[final_dataset['membership'] == 'Gold'])
gold_percentage = (gold_orders / total_orders) * 100

print(f"Total Orders: {total_orders}")
print(f"Gold Member Orders: {gold_orders}")
print(f"Percentage: {gold_percentage:.2f}%")
print(f"Rounded: {round(gold_percentage)}%")

print(f"\n✓ ANSWER: {round(gold_percentage)}%")

In [ ]:
membership_cuisine_revenue = final_dataset.groupby(['membership', 'cuisine'])['total_amount'].sum().sort_values(ascending=False)

print("Top Membership + Cuisine Combinations by Revenue:")
for (membership, cuisine), revenue in membership_cuisine_revenue.head(10).items():
    print(f"{membership:8} + {cuisine:10} : ₹{revenue:,.2f}")

top_combo = membership_cuisine_revenue.idxmax()
print(f"\n✓ ANSWER: {top_combo[0]} + {top_combo[1]} cuisine")

In [ ]:
quarterly_revenue = final_dataset.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)

quarter_names = {1: 'Q1 (Jan-Mar)', 2: 'Q2 (Apr-Jun)',
                 3: 'Q3 (Jul-Sep)', 4: 'Q4 (Oct-Dec)'}

print("Quarterly Revenue:")
for quarter, revenue in quarterly_revenue.items():
    print(f"{quarter_names[quarter]}: ₹{revenue:,.2f}")

print(f"\n✓ ANSWER: {quarter_names[quarterly_revenue.idxmax()]}")

In [ ]:
gold_orders_count = len(final_dataset[final_dataset['membership'] == 'Gold'])
print(f"✓ ANSWER: {gold_orders_count}")

In [ ]:
hyderabad_revenue = final_dataset[final_dataset['city'] == 'Hyderabad']['total_amount'].sum()
hyderabad_revenue_rounded = round(hyderabad_revenue)
print(f"Total revenue from Hyderabad: ₹{hyderabad_revenue:.2f}")
print(f"✓ ANSWER: {hyderabad_revenue_rounded}")

In [ ]:
distinct_users = final_dataset['user_id'].nunique()
print(f"✓ ANSWER: {distinct_users}")

In [ ]:
gold_avg_order = final_dataset[final_dataset['membership'] == 'Gold']['total_amount'].mean()
gold_avg_rounded = round(gold_avg_order, 2)
print(f"✓ ANSWER: {gold_avg_rounded}")

In [ ]:
orders_rating_4_5_plus = len(final_dataset[final_dataset['rating'] >= 4.5])
print(f"✓ ANSWER: {orders_rating_4_5_plus}")

In [ ]:
gold_df = final_dataset[final_dataset['membership'] == 'Gold']
gold_city_revenue = gold_df.groupby('city')['total_amount'].sum().sort_values(ascending=False)
top_city = gold_city_revenue.idxmax()
orders_in_top_city = len(gold_df[gold_df['city'] == top_city])

print(f"Top revenue city for Gold members: {top_city}")
print(f"✓ ANSWER: {orders_in_top_city}")

In [ ]:
print("="*80)
print("FILL-IN-THE-BLANK ANSWERS")
print("="*80)

answers = {
    "Q1: Column used to join orders.csv and users.json": "user_id",
    "Q2: Dataset containing cuisine and rating information format": "SQL",
    "Q3: Total number of rows in final merged dataset": len(final_dataset),
    "Q4: Merged values when user has no matching record": "NULL or NaN",
    "Q5: Pandas function used to combine datasets": "merge",
    "Q6: Column membership originates from file": "users.json",
    "Q7: Join key to combine orders with restaurant details": "restaurant_id",
    "Q8: Column identifying type of food served": "cuisine",
    "Q9: User details appear ___ times for multiple orders": "multiple"
}

for question, answer in answers.items():
    print(f"\n{question}")
    print(f"✓ ANSWER: {answer}")

print("\n" + "="*80)

In [ ]:
customer_ltv = final_dataset.groupby('user_id').agg({
    'total_amount': 'sum',
    'order_id': 'count'
}).round(2)
customer_ltv.columns = ['Lifetime_Value', 'Total_Orders']
customer_ltv['Avg_Order_Value'] = (customer_ltv['Lifetime_Value'] / customer_ltv['Total_Orders']).round(2)

print("Customer Lifetime Value Statistics:")
print(f"Average Customer LTV: ₹{customer_ltv['Lifetime_Value'].mean():.2f}")
print(f"Median Customer LTV: ₹{customer_ltv['Lifetime_Value'].median():.2f}")
print(f"Top 10% Customer LTV: ₹{customer_ltv['Lifetime_Value'].quantile(0.9):.2f}")

print("\nTop 10 Customers by Lifetime Value:")
display(customer_ltv.sort_values('Lifetime_Value', ascending=False).head(10))

In [ ]:
# Revenue distribution by order value brackets
revenue_bins = [0, 250, 500, 750, 1000, 1500, float('inf')]
revenue_labels = ['₹0-250', '₹251-500', '₹501-750', '₹751-1000', '₹1001-1500', '₹1500+']
final_dataset['order_value_bracket'] = pd.cut(final_dataset['total_amount'],
                                               bins=revenue_bins,
                                               labels=revenue_labels)

revenue_distribution = final_dataset.groupby('order_value_bracket', observed=True).agg({
    'order_id': 'count',
    'total_amount': 'sum'
}).round(2)
revenue_distribution.columns = ['Order_Count', 'Total_Revenue']
revenue_distribution['Percentage_of_Orders'] = (revenue_distribution['Order_Count'] / len(final_dataset) * 100).round(2)
revenue_distribution['Percentage_of_Revenue'] = (revenue_distribution['Total_Revenue'] / final_dataset['total_amount'].sum() * 100).round(2)

print("Revenue Distribution by Order Value Bracket:")
display(revenue_distribution)

In [ ]:
plt.figure(figsize=(14, 6))

monthly_data = final_dataset.groupby(final_dataset['order_date'].dt.to_period('M')).agg({
    'total_amount': 'sum'
}).reset_index()
monthly_data['order_date'] = monthly_data['order_date'].astype(str)

plt.plot(monthly_data['order_date'], monthly_data['total_amount'],
         marker='o', linewidth=2, markersize=8, color='#2E86AB')
plt.title('Monthly Revenue Trend', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Total Revenue (₹)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

city_revenue = final_dataset.groupby('city')['total_amount'].sum().sort_values(ascending=False)

colors = ['#A23B72', '#F18F01', '#C73E1D', '#6A994E']
bars = plt.bar(city_revenue.index, city_revenue.values, color=colors, alpha=0.8, edgecolor='black')

plt.title('Total Revenue by City', fontsize=16, fontweight='bold')
plt.xlabel('City', fontsize=12)
plt.ylabel('Total Revenue (₹)', fontsize=12)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'₹{height:,.0f}',
             ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Revenue by Cuisine
cuisine_revenue = final_dataset.groupby('cuisine')['total_amount'].sum().sort_values(ascending=False)
axes[0].bar(cuisine_revenue.index, cuisine_revenue.values,
            color=['#E63946', '#F1FAEE', '#A8DADC', '#457B9D'], alpha=0.8, edgecolor='black')
axes[0].set_title('Total Revenue by Cuisine', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Cuisine', fontsize=11)
axes[0].set_ylabel('Total Revenue (₹)', fontsize=11)
axes[0].tick_params(axis='x', rotation=0)

# Average Order Value by Cuisine
cuisine_avg = final_dataset.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)
axes[1].bar(cuisine_avg.index, cuisine_avg.values,
            color=['#E63946', '#F1FAEE', '#A8DADC', '#457B9D'], alpha=0.8, edgecolor='black')
axes[1].set_title('Average Order Value by Cuisine', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Cuisine', fontsize=11)
axes[1].set_ylabel('Average Order Value (₹)', fontsize=11)
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Orders Count
membership_orders = final_dataset.groupby('membership')['order_id'].count()
axes[0].bar(membership_orders.index, membership_orders.values,
            color=['#FFD700', '#C0C0C0'], alpha=0.8, edgecolor='black', width=0.6)
axes[0].set_title('Total Orders by Membership', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Number of Orders', fontsize=11)

# Revenue
membership_revenue = final_dataset.groupby('membership')['total_amount'].sum()
axes[1].bar(membership_revenue.index, membership_revenue.values,
            color=['#FFD700', '#C0C0C0'], alpha=0.8, edgecolor='black', width=0.6)
axes[1].set_title('Total Revenue by Membership', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Total Revenue (₹)', fontsize=11)

# Average Order Value
membership_avg = final_dataset.groupby('membership')['total_amount'].mean()
axes[2].bar(membership_avg.index, membership_avg.values,
            color=['#FFD700', '#C0C0C0'], alpha=0.8, edgecolor='black', width=0.6)
axes[2].set_title('Average Order Value by Membership', fontsize=13, fontweight='bold')
axes[2].set_ylabel('Average Order Value (₹)', fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

quarterly_revenue = final_dataset.groupby('quarter')['total_amount'].sum()
quarter_labels = ['Q1\n(Jan-Mar)', 'Q2\n(Apr-Jun)', 'Q3\n(Jul-Sep)', 'Q4\n(Oct-Dec)']

colors_q = ['#264653', '#2A9D8F', '#E9C46A', '#F4A261']
bars = plt.bar(quarter_labels, quarterly_revenue.values, color=colors_q, alpha=0.8, edgecolor='black')

plt.title('Quarterly Revenue Distribution', fontsize=16, fontweight='bold')
plt.ylabel('Total Revenue (₹)', fontsize=12)
plt.xlabel('Quarter', fontsize=12)

# Add value labels
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'₹{height:,.0f}',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

plt.hist(final_dataset['rating'], bins=20, color='#06D6A0', alpha=0.7, edgecolor='black')
plt.axvline(final_dataset['rating'].mean(), color='red', linestyle='--',
            linewidth=2, label=f'Mean: {final_dataset["rating"].mean():.2f}')
plt.axvline(final_dataset['rating'].median(), color='orange', linestyle='--',
            linewidth=2, label=f'Median: {final_dataset["rating"].median():.2f}')

plt.title('Distribution of Restaurant Ratings', fontsize=16, fontweight='bold')
plt.xlabel('Rating', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.hist(final_dataset['total_amount'], bins=50, color='#EF476F', alpha=0.7, edgecolor='black')
plt.axvline(final_dataset['total_amount'].mean(), color='blue', linestyle='--',
            linewidth=2, label=f'Mean: ₹{final_dataset["total_amount"].mean():.2f}')
plt.title('Distribution of Order Values', fontsize=14, fontweight='bold')
plt.xlabel('Order Value (₹)', fontsize=11)
plt.ylabel('Frequency', fontsize=11)
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot([final_dataset[final_dataset['membership']=='Gold']['total_amount'],
             final_dataset[final_dataset['membership']=='Regular']['total_amount']],
            labels=['Gold', 'Regular'],
            patch_artist=True,
            boxprops=dict(facecolor='lightblue', alpha=0.7),
            medianprops=dict(color='red', linewidth=2))
plt.title('Order Value Distribution by Membership', fontsize=14, fontweight='bold')
plt.ylabel('Order Value (₹)', fontsize=11)
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
print("="*80)
print("KEY FINDINGS & INSIGHTS")
print("="*80)

print("\n📊 DATASET SUMMARY:")
print(f"  • Total Orders: {len(final_dataset):,}")
print(f"  • Total Revenue: ₹{final_dataset['total_amount'].sum():,.2f}")
print(f"  • Unique Users: {final_dataset['user_id'].nunique():,}")
print(f"  • Unique Restaurants: {final_dataset['restaurant_id'].nunique():,}")

print("\n🏙️ TOP PERFORMING CITY:")
top_city = final_dataset.groupby('city')['total_amount'].sum().idxmax()
top_city_revenue = final_dataset.groupby('city')['total_amount'].sum().max()
print(f"  • {top_city}: ₹{top_city_revenue:,.2f}")

print("\n🍽️ MOST POPULAR CUISINE:")
top_cuisine = final_dataset['cuisine'].value_counts().idxmax()
top_cuisine_orders = final_dataset['cuisine'].value_counts().max()
print(f"  • {top_cuisine}: {top_cuisine_orders:,} orders")

print("\n💎 MEMBERSHIP INSIGHTS:")
gold_pct = (len(final_dataset[final_dataset['membership']=='Gold']) / len(final_dataset)) * 100
print(f"  • Gold Members: {gold_pct:.1f}% of orders")
print(f"  • Regular Members: {100-gold_pct:.1f}% of orders")

print("\n📈 SEASONALITY:")
peak_quarter = final_dataset.groupby('quarter')['total_amount'].sum().idxmax()
quarter_names = {1: 'Q1 (Jan-Mar)', 2: 'Q2 (Apr-Jun)', 3: 'Q3 (Jul-Sep)', 4: 'Q4 (Oct-Dec)'}
print(f"  • Peak Quarter: {quarter_names[peak_quarter]}")

print("\n⭐ QUALITY METRICS:")
print(f"  • Average Restaurant Rating: {final_dataset['rating'].mean():.2f}")
print(f"  • Average Order Value: ₹{final_dataset['total_amount'].mean():.2f}")

print("\n" + "="*80)
print("✓ ANALYSIS COMPLETE")
print("="*80)